In [1]:
pip install google-auth
pip install google-auth-oauthlib
!pip install google-auth-httplib2
!pip install google-api-python-client
!pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [44]:
import google.auth
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
import tempfile
import pandas as pd
import openpyxl
from datetime import datetime, timedelta


with open(r'C:\Users\IAN NAMBOGA MADETE\Downloads\boxwood-chassis-438812-q4-da810f36cfed.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
with tempfile.NamedTemporaryFile(mode='w', encoding='utf-8', delete=False) as temp_file:
    temp_file.write(json.dumps(data))
creds = service_account.Credentials.from_service_account_file(temp_file.name, scopes =['https://www.googleapis.com/auth/webmasters.readonly'])


def get_search_console_service():
    """Build and return the Search Console service"""
    try:
        service = build('searchconsole', 'v1', credentials=creds)
        return service
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

    
def get_site_list(service):
    """Get list of sites in Search Console"""
    try:
        sites = service.sites().list().execute()
        return sites
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

def get_leading_queries(service, site_url):
    """Get leading queries for each URL"""
    try:
        # Set date range (last 28 days)
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=28)).strftime('%Y-%m-%d')
        
        # Initialize results list
        results = []
        
        # Request body for Search Analytics
        request = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': ['page', 'query'],
            'rowLimit': 25000,
            'startRow': 0
        }
        
        response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()
        
        # Process URLs and their queries
        url_data = {}
        
        if 'rows' in response:
            for row in response['rows']:
                url = row['keys'][0]
                query = row['keys'][1]
                clicks = row.get('clicks', 0)
                impressions = row.get('impressions', 0)
                
                if url not in url_data:
                    url_data[url] = {
                        'queries_clicks': [],
                        'queries_impressions': []
                    }
                
                url_data[url]['queries_clicks'].append((query, clicks))
                url_data[url]['queries_impressions'].append((query, impressions))
            
            # Find leading queries for each URL
            for url, data in url_data.items():
                # Sort queries by clicks and impressions
                clicks_sorted = sorted(data['queries_clicks'], 
                                    key=lambda x: x[1], 
                                    reverse=True)
                impressions_sorted = sorted(data['queries_impressions'], 
                                         key=lambda x: x[1], 
                                         reverse=True)
                
                # Get leading queries
                lead_query_clicks = clicks_sorted[0][0] if clicks_sorted and clicks_sorted[0][1] > 0 else "-"
                lead_query_impressions = impressions_sorted[0][0] if impressions_sorted else "-"
                
                results.append({
                    'URL': url,
                    'Leading Query (Clicks)': lead_query_clicks,
                    'Leading Query (Impressions)': lead_query_impressions
                })
        
        return results
    
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

def export_to_excel(results, filename):
    """Export results to Excel file"""
    try:
        df = pd.DataFrame(results)
        df.to_excel(filename, index=False)
        print(f"Data successfully exported to {filename}")
    except Exception as error:
        print(f'An error occurred while exporting to Excel: {error}')


def main():
    # Get Search Console service
    service = get_search_console_service()
    if not service:
        return
    
    # Get list of sites
    sites_data = get_site_list(service)
    if not sites_data:
        return
    
    print("\nAvailable sites:")
    for site in sites_data.get('siteEntry', []):
        print(f"- {site['siteUrl']}")
    
    selected_site = input("\nEnter the complete site URL from above: ")
    
    results = get_leading_queries(service, selected_site)
    if not results:
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'leading_queries_{timestamp}.xlsx'
    export_to_excel(results, filename)

if __name__ == "__main__":
    main()
    
